In [ ]:
import zipfile

zip_path = "/home/swc/Downloads/coords_Non.zip"

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
extract = "yolov9/data"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract)
print("Done")

Done


In [12]:
import pandas as pd

df= pd.read_csv("/content/drive/MyDrive/Data_Kyutech/coords_Non.csv")

In [13]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z520,v520,x521,y521,z521,v521,x522,y522,z522,v522
0,Hold,0.407733,0.129086,-0.778690,0.993342,0.421445,0.090329,-0.737180,0.993019,0.434254,...,-0.078738,0.0,0.911478,0.763253,-0.085874,0.0,0.918760,0.787047,-0.091987,0.0
1,Hold,0.405080,0.119946,-0.982599,0.993911,0.419926,0.084866,-0.928936,0.993197,0.433784,...,-0.081532,0.0,0.907795,0.763564,-0.088705,0.0,0.912692,0.788697,-0.094718,0.0
2,Hold,0.397699,0.109015,-0.836984,0.994504,0.419621,0.072088,-0.792933,0.993758,0.433126,...,-0.077996,0.0,0.909169,0.768035,-0.084063,0.0,0.914087,0.792701,-0.089030,0.0
3,Hold,0.395727,0.105420,-0.799126,0.994938,0.418676,0.067525,-0.756440,0.993735,0.432624,...,-0.073175,0.0,0.910434,0.768192,-0.078608,0.0,0.918212,0.791781,-0.082382,0.0
4,Hold,0.394592,0.100825,-0.968101,0.995398,0.418038,0.063968,-0.916498,0.994151,0.432285,...,-0.080001,0.0,0.810127,0.742485,-0.083544,0.0,0.814252,0.768813,-0.084207,0.0


In [14]:
%pip install tensorflow

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

def process_data(file_path, frames=30, test_size=0.2, random_state=42):
    print("Processing data")

    # Load the DataFrame from file
    df = pd.read_csv(file_path)

    # Debugging: Check the type of df after loading it
    print(f"Type of df after loading: {type(df)}")

    # Ensure that df is a pandas DataFrame
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected a pandas DataFrame, but got {type(df)}")

    # Print the first few rows of the DataFrame to ensure it's correct
    print(f"First 5 rows of the data:\n{df.head()}")

    # Extract class column (assuming first column is the class label)
    class_column = df.columns[0]
    print(f"Class column: {class_column}")

    # Get all labels
    all_labels = df[class_column].values

    # Encode all labels using LabelEncoder
    label_encoder = LabelEncoder()
    all_numeric_labels = label_encoder.fit_transform(all_labels)
    class_names = label_encoder.classes_
    print(f"Found {len(class_names)} classes: {class_names}")

    # Drop the class column to get features
    features_df = df.drop(columns=[class_column])
    num_features = features_df.shape[1]
    print(f"Total features: {num_features}")

    # Check for missing values
    missing_values = features_df.isnull().sum().sum()
    print(f"Missing values: {missing_values}")
    if missing_values > 0:
        features_df.fillna(0, inplace=True)
        print("Filled missing values with 0")

    # Normalize the feature data
    print("Normalizing data")
    scaler = MinMaxScaler()
    normalized_features = scaler.fit_transform(features_df)

    # Organize data into sequences
    print(f"Organizing data into {frames}-frame sequences")
    num_frames = len(normalized_features)
    num_complete_sequences = num_frames // frames
    total_frames_used = num_complete_sequences * frames

    trimmed_features = normalized_features[:total_frames_used]
    trimmed_labels = all_numeric_labels[:total_frames_used]

    print(f"Using {total_frames_used} frames out of {num_frames} total frames")
    print(f"Creating {num_complete_sequences} complete sequences")

    # Prepare the sequences (X) and labels (y)
    X_sequences = np.zeros((num_complete_sequences, frames, num_features))
    y_sequences = np.zeros(num_complete_sequences)

    for i in range(num_complete_sequences):
        start = i * frames
        end = start + frames
        X_sequences[i] = trimmed_features[start:end]

        # Get labels for the current sequence (use most common label)
        sequence_labels = trimmed_labels[start:end]
        unique_labels, counts = np.unique(sequence_labels, return_counts=True)
        if len(unique_labels) > 0:
            most_common_idx = np.argmax(counts)
            y_sequences[i] = unique_labels[most_common_idx]
        else:
            y_sequences[i] = 0

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_sequences, y_sequences, test_size=test_size, random_state=random_state
    )

    # Print data shapes
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")

    # Return all relevant data
    return X_train, X_test, y_train, y_test, class_names, scaler, label_encoder, num_features, frames, len(class_names)



In [55]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_sequence(sequence, class_name, num_frames_to_show=5):

    total_frames = sequence.shape[0]

    # Select frames to visualize (evenly spaced)
    frame_indices = np.linspace(0, total_frames - 1, num_frames_to_show, dtype=int)

    fig, axes = plt.subplots(1, num_frames_to_show, figsize=(15, 3))
    fig.suptitle(f'Gesture: {class_name}', fontsize=16)

    for i, frame_idx in enumerate(frame_indices):
        frame = sequence[frame_idx]

        # Assuming 21 landmarks per hand (x, y, z)
        num_landmarks = 21
        hand_features = frame[:num_landmarks * 3].reshape(num_landmarks, 3)

        # Extract x, y coordinates for visualization
        x_coords = hand_features[:, 0]
        y_coords = hand_features[:, 1]

        # Plot hand landmarks
        axes[i].scatter(x_coords, y_coords, color='blue', s=30)
        axes[i].set_xlim(0, 1)
        axes[i].set_ylim(1, 0)  # Flip y-axis to match image coordinates
        axes[i].set_title(f'Frame {frame_idx}')
        axes[i].set_xticks([])
        axes[i].set_yticks([])

    plt.tight_layout()
    plt.show()


In [62]:
if __name__ == "__main__":
    data_path = "/content/drive/MyDrive/Data_Kyutech/coords_Non.csv"  # Correct CSV file path

    try:
        processed_data = process_data(data_path, frames=30)
        data = process_data(df, frames=30)
          X_train = data['X_train']
          X_test = data['X_test']
          y_train = data['y_train']
          y_test = data['y_test']
          class_names = data['class_names']
          scaler = data['scaler']


        # Print summary info
        print("\nData Processing Summary:")
        print(f"Number of gesture classes: {processed_data['num_classes']}")
        print(f"Class names: {processed_data['class_names']}")
        print(f"Features per frame: {processed_data['num_features']}")
        print(f"Frames per gesture: {processed_data['frames_per_gesture']}")
        print(f"Training sequences: {processed_data['X_train'].shape[0]}")
        print(f"Test sequences: {processed_data['X_test'].shape[0]}")

        # Visualize a sample sequence
        sample_idx = 0
        class_idx = int(processed_data['y_train'][sample_idx])
        class_name = processed_data['class_names'][class_idx]
        print(f"\nVisualizing sample sequence (class: {class_name})...")
        visualize_sequence(processed_data['X_train'][sample_idx], class_name)

    except Exception as e:
        print(f"Error processing data: {e}")


Processing data
Type of df after loading: <class 'pandas.core.frame.DataFrame'>
First 5 rows of the data:
  class        x1        y1        z1        v1        x2        y2        z2  \
0  Hold  0.407733  0.129086 -0.778690  0.993342  0.421445  0.090329 -0.737180   
1  Hold  0.405080  0.119946 -0.982599  0.993911  0.419926  0.084866 -0.928936   
2  Hold  0.397699  0.109015 -0.836984  0.994504  0.419621  0.072088 -0.792933   
3  Hold  0.395727  0.105420 -0.799126  0.994938  0.418676  0.067525 -0.756440   
4  Hold  0.394592  0.100825 -0.968101  0.995398  0.418038  0.063968 -0.916498   

         v2        x3  ...      z520  v520      x521      y521      z521  \
0  0.993019  0.434254  ... -0.078738   0.0  0.911478  0.763253 -0.085874   
1  0.993197  0.433784  ... -0.081532   0.0  0.907795  0.763564 -0.088705   
2  0.993758  0.433126  ... -0.077996   0.0  0.909169  0.768035 -0.084063   
3  0.993735  0.432624  ... -0.073175   0.0  0.910434  0.768192 -0.078608   
4  0.994151  0.432285  ... 

In [57]:
from re import A
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam

def build_bilstm_model(input, classes):
  model = Sequential([
                Input(shape=input_shape),
                Bidirectional(LSTM(128, return_sequences = True, activation = "relu")),
                BatchNormalization(),
                Dropout(0.3),

                Bidirectional(LSTM(64, return_sequences = True, activation = "relu")),
                BatchNormalization(),
                Dropout(0.3),

                Bidirectional(LSTM(32, return_sequences = True, activation = "relu")),
                BatchNormalization(),
                Dropout(0.3),

                Dense(64, activation = "relu"),
                BatchNormalization(),
                Dropout(0.3),

                Dense(32, activation = "relu"),
                BatchNormalization(),
                Dropout(0.3),

                Dense(classes, activation = "softmax")])
  return model




In [58]:
input_shape = (30,2088)
num_classes = 25
model = build_bilstm_model(input_shape, num_classes)
bilstm_model = build_bilstm_model(input_shape, num_classes)

# Compile model
bilstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
bilstm_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_24 (Bidirectional)     │ (None, 30, 256)             │       2,270,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_40               │ (None, 30, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_40 (Dropout)                 │ (None, 30, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_25 (Bidirectional)     │ (None, 30, 128)             │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_41               │ (None, 30, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_41 (Dropout)                 │ (None, 30, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_26 (Bidirectional)     │ (None, 30, 64)              │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_42               │ (None, 30, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_42 (Dropout)                 │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 30, 64)              │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_43               │ (None, 30, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_43 (Dropout)                 │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 30, 32)              │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_44               │ (None, 30, 32)              │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_44 (Dropout)                 │ (None, 30, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 30, 25)              │             825 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,485,017 (9.48 MB)

 Trainable params: 2,483,929 (9.48 MB)

 Non-trainable params: 1,088 (4.25 KB)

In [59]:
X_train, X_test, y_train, y_test, class_names, scaler, label_encoder, num_features, frames, num_classes = process_data(data_path, frames=30)

# Check the type of X_train
print(f"Type of X_train after process_data: {type(X_train)}")

# Check if X_train is really a string
if isinstance(X_train, str):
    print("X_train is a string, let's debug further...")
else:
    print(f"Shape of X_train: {X_train.shape}")  # If it's a NumPy array, print shape
    print(f"First sample in X_train:\n{X_train[0]}")


Processing data


TypeError: argument of type 'method' is not iterable

In [60]:
print(X_train)

X_train


In [50]:
X_train

'X_train'

In [30]:
history = bilstm_model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))


ValueError: Unrecognized data type: x=X_train (of type <class 'str'>)